In [ ]:
!pip install ultralytics
!pip install split-folders

In [ ]:
!nvidia-smi

In [ ]:
import os
import shutil
from ultralytics import YOLO  # For loading and running YOLOv8 models
from pathlib import Path
import torch  # For working with PyTorch tensors (used by YOLOv8)
import cv2  # For image and video processinga
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For visualizing results
import splitfolders
import random
import pandas as pd
import seaborn as sns
from PIL import Image
import imghdr 

In [ ]:
trash_dir = '/kaggle/input/garbage-classification-v2/garbage-dataset/trash/'
paper_dir = '/kaggle/input/garbage-classification-v2/garbage-dataset/paper/'
metal_dir = '/kaggle/input/garbage-classification-v2/garbage-dataset/metal/'

In [ ]:
def remove_folder_contents(folder):
    """Removes all files and subdirectories inside the given folder."""
    for item in os.listdir(folder):
        item_path = os.path.join(folder, item)
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)  # Remove file or symbolic link
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)  # Remove entire directory and its contents
        except Exception as e:
            print(f"Error deleting {item_path}: {e}")

# Usage
folder_path = '/kaggle/working'
remove_folder_contents(folder_path)

In [ ]:
# shutil.copytree(data_dir, '/kaggle/working/dataset')
try:
        shutil.copytree(trash_dir, '/kaggle/working/dataset/trash')
        shutil.copytree(paper_dir, '/kaggle/working/dataset/paper')
        shutil.copytree(metal_dir, '/kaggle/working/dataset/metal')
except Exception as e:
    print(e)
    pass

In [ ]:
data_dir  = '/kaggle/working/dataset'
experiment_name = "Transfer Learning"

In [ ]:
#TODO make sure each class has roughly the same amount of images

In [ ]:
#TODO double check what image formats are supported by yolo
count=0
image_extensions = [".png", ".jpg"] 
img_type_accepted_by_ul = ["bmp", "gif", "jpeg", "png"]

for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        if img_type not in img_type_accepted_by_ul:
            print(f"{filepath} is a {img_type}, not accepted by ultralytics")
            os.remove(filepath)
            count+=1
print(f"Removed {count} images")

In [ ]:
#TODO add image labels (idk how)

In [ ]:
#TODO apply image transformations (i.e. resizing, cropping, random rotations, brightness/contrast adjustments, gaussian noise, normalization)

In [ ]:
os.mkdir('data')
os.chdir('data')
os.getcwd()
os.mkdir('train')
os.mkdir('test')
os.mkdir('val')
os.chdir('/kaggle/working/')
os.getcwd()

In [ ]:
# splitfolders.ratio("/kaggle/working/dataset", output = 'data', seed = 42, ratio = (.8,.1,.1), group_prefix = None)

In [ ]:
#from ultralytics import YOLO
#from ultralytics.data.utils import autosplit 

##TODO switch to different splitting function that preserves class ratios when splitting (StratifiedShuffleSplit from sk-learn)
#autosplit(  
#    path="/kaggle/working/dataset/",
#    weights=(0.8, 0.1, 0.1),  # (train, validation, test) fractional splits
#    annotated_only=False,  # split only images with annotation file when True
#)

In [ ]:
#This is not currently working

#yaml_content = """path: /kaggle/working/  # Base directory for dataset
#train: autosplit_train.txt  # Training dataset folder
#val: autosplit_val.txt  # Validation dataset folder
#test: autosplit_test.txt
#nc: 3  # Number of classes (change accordingly)
#names: ["metal", "paper", "trash"]  # Class names
#"""

# Write the content to a YAML file
#with open("/kaggle/working/dataset.yaml", "w") as file:
#    file.write(yaml_content)

#print("YAML file created successfully!")


In [ ]:
splitfolders.ratio("/kaggle/working/dataset", output = 'data', seed = 42, ratio = (.8,.1,.1), group_prefix = None)

In [ ]:
print(os.listdir("/kaggle/working/"))  # Should include "dataset.yaml"

In [ ]:
from ultralytics import YOLO

# Load a YOLOv8 model
model = YOLO("yolo11n-cls.pt")  # load a pretrained yolo11n-cls.ptd model (recommended for training)

# Train the model
results = model.train(data="/kaggle/working/data", epochs=1, imgsz=600, device=[0,1])


In [ ]:
# Export the model to ONNX (Change to needed format) 
#model.export(format="onnx")

torch.save(model.state_dict(), 'yolo_model.pth')
torch.save(model, 'full_yolo_model.pth')
model.export(format='onnx')  # This will export the model as an ONNX file.

print("Exported")

In [ ]:
#Validate
#metrics = model.val()  # no arguments needed, uses the dataset and settings from training
#metrics.top1  # top1 accuracy
#metrics.top5  # top5 accuracy

import yaml

data_dict = {
    'train': '/kaggle/working/data/train',
    'val': '/kaggle/working/data/train',
    'nc': 3,  # number of classes
    'names': ['trash', 'metal', 'paper']
}

# Save it as a YAML file
with open('data.yaml', 'w') as f:
    yaml.dump(data_dict, f)

# Load a trained model
model = YOLO("/kaggle/working/runs/classify/train/weights/best.pt")
#model = YOLO("/kaggle/working/yolo11n.pt")  # e.g., "yolov8n.pt" or "runs/train/exp/weights/best.pt"
results = model.val(data="data.yaml")
